In [ ]:
# from stable_baselines3 import PPO

In [1]:
from queue import Queue
from threading import Thread
import threading
import gym
from gym import spaces,Env
import sys

In [2]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [3]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [4]:
from feed_env import Episode

importing Jupyter notebook from feed_env.ipynb


In [5]:
import aspectlib
import gym
from gym import spaces,Env

In [6]:
from aiagentbase import AIAgent,Controller,Memory,Perception,Actor,RLAgent

importing Jupyter notebook from aiagentbase.ipynb


In [7]:
from featfuncs import USE_COLS_DICT_ORIG,update_use_cols_dict
GDIM=3

In [8]:
# OHLCV_COLS=['Open_n','High_n','Low_n','Close_n','Volume_n']
# TA_COLS=['SMA_10', 'SMA_20','VOL_SMA_20','RSI_14','BBL_5_2.0','BBM_5_2.0','BBU_5_2.0',
#        'BBB_5_2.0', 'BBP_5_2.0','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','VWAP_D',
#         'MOM_30','CMO_14']
# TA_COLS_MIN=['SMA_10', 'SMA_20','CMO_14']
# LOGICAL_FEATURES=['Open_changelen',
#  'High_changelen',
#  'Low_changelen',
#  'Close_changelen',
#  'High-Low_changelen',
#  'Open-Close_changelen',
#  'Open_changelen_val',
#  'Open_changelen_polarity',
#  'High_changelen_val',
#  'High_changelen_polarity',
#  'Low_changelen_val',
#  'Low_changelen_polarity',
#  'Close_changelen_val',
#  'Close_changelen_polarity',
#  'High-Low_changelen_val',
#  'High-Low_changelen_polarity',
#  'Open-Close_changelen_val',
#  'Open-Close_changelen_polarity']
# # COLS=['row_num']+OHLCV_COLS+TA_COLS
# # COLS=['row_num']+OHLCV_COLS+TA_COLS_MIN
# # DIM=len(COLS)
# GDIM=3
# MINCOLS=['row_num']+OHLCV_COLS+TA_COLS_MIN
# ALLCOLS=['row_num']+OHLCV_COLS+TA_COLS
# MINLOG=['row_num']+OHLCV_COLS+TA_COLS_MIN+LOGICAL_FEATURES
# ALLLOG=['row_num']+OHLCV_COLS+TA_COLS_MIN+LOGICAL_FEATURES
# LOGCOLS=['row_num']+OHLCV_COLS+LOGICAL_FEATURES
# USE_COLS_DICT={'allcols':ALLCOLS,'mincols':MINCOLS,'minlog':MINLOG,
#            'alllog':ALLLOG,'logcols':LOGCOLS}

### Trading strategies as agents
Interface between backtest and tradeserver. TBD: common rewards format
Note: only works for local strategies (not remote: TBD).

In [9]:
class StratAgent(AIAgent):
    def __init__(self,use_cols=None):
        self.agent=True
        self.tidx=0
        self.owner=None
        super().__init__()
        self.use_alt_data=False
        self.use_memory=False
        # self.actor=self.Actor(parent=self)
        self.logL=[]
        self.action_space=spaces.Discrete(3)
        if use_cols:
            USE_COLS_DICT=update_use_cols_dict(USE_COLS_DICT_ORIG)
            self.feat_cols=USE_COLS_DICT[use_cols]
            self.dim=len(self.feat_cols)
            self.data_cols=['datetime']+self.feat_cols
        self.model_type='RL'
        self.actor.call_model=lambda state: self.call_model(state)
        self.actor.percept_to_state=self.percept_to_state
    def percept_to_state(self,perceived_state):
        # print(perceived_state)
        percept=perceived_state['percept']
        state=torch.tensor(percept[self.feat_cols].iloc[-1].values)
        return state
    def initialize(self):
        self.agent=True
        self.tidx=0
        self.owner=None
        self.use_memory=False
        self.logL=[]
        self.action_space=spaces.Discrete(3)
        self.data_cols=['datetime']+self.feat_cols
        # self.data_cols=['datetime']+COLS
        self.model_type='RL'
    ##Interface to tradeserver
    def compress_alt_data(self,gdata):
        gstateD={}
        cD={}
        dD={}
        for d in gdata:
            gd=gdata[d][0]
            v,m=0,0
            for gt in ['^NSEI','^NYA','LSEG.L','^IXIC']:
                if 'Open_'+gt in gd:
                    cD[gt]=gd['Close_'+gt]-gd['Open_'+gt]
                    dD[gt]=gd['High_'+gt]-gd['Low_'+gt]
                    v+=gd['Volume_'+gt]
                    m+=1
                else: cD[gt],dD[gt]=0,0
            v=v/m
            gstateD[d]=torch.tensor([sum([cD[c] for c in cD])/m,sum([dD[d] for d in dD])/m,v])
            gstateD[d]=torch.nan_to_num(gstateD[d])
        return gstateD
    def set_alt_data(self,alt_data_func,remote=False):
        self.use_alt_data=True
        if remote: self.gdata=anvil.server.call(alt_data_func)['gdata']
        else: self.gdata=alt_data_func()['gdata']
        self.gstateD=self.compress_alt_data(self.gdata)
        # self.gstateD={d: torch.tensor(pd.DataFrame(self.gdata[d]).iloc[-1].values)
        #              for d in self.gdata}
    def act_on_entry(self):
        if self.owner==None: return True
        elif self.owner.status[self.scantickers[self.tidx]]=='deployed': return False
        else: return True
    def act_on_exit(self):
        if self.owner==None: return True
        elif self.owner.status[self.scantickers[self.tidx]]=='active': return False
        else: return True
    def check_entry_batch(self,dfD):
        decisionsD,stopD,targetD=self.check_entry(dfD)
        if self.act_on_entry(): 
            trt=self.scantickers[self.tidx]
            df=dfD[trt]
            action=self.act(df)
            decisionsD[trt]=action-1
        return decisionsD,stopD,targetD
    def save_func(self,episode_state):
        return ()
    def check_exit_batch(self,dfD,posf):
        return posf
    def exit_predicate(self,row,df):
        return False
    def Check(strat,dfD):
        return strat.check_entry_batch(dfD)
    def Exit(strat,dfD,posf):
        return strat.check_exit_batch(dfD,posf)
    def check_entry(self,dfD):
        decisionsD={t:0 for t in dfD}
        stopD={t:0 for t in dfD}
        targetD={t:0 for t in dfD}
        return decisionsD,stopD,targetD
    def exit_func(self,row,df):
        return False
    def save_func(self,episode_state):
        ticker=[t for t in episode_state][0]
        return ticker,self.entry_val[ticker],self.exit_val[ticker]

In [ ]:
class RandomStratAgent(StratAgent):
    # Adaptive momentum strategy using CMO and ADF for non-stationarity
    def __init__(self):
        # super().__init__()
        StratAgent.__init__(self)
        self.logL=[]
        self.logL=[]
        self.rewL=[]
        ## Augmentations of AIAgent
        self.perception.perceive_reward=self.perceive_reward
    def call_model(self,state):
        # super().act(state)
        return self.action_space.sample()
    def perceive_reward(self,reward):
        return reward[0]

In [ ]:
class RLStratAgentDyn(RLAgent,StratAgent):
    def __init__(self,algoclass,soclass,monclass,myargs=(2048,False,5,'mincols'),verbose=1,
                 metarl=True):
        use_cols=myargs[3]
        action_space=spaces.MultiDiscrete([3,3,3])
        n_steps,self.use_alt_data,win=myargs[0],myargs[1],myargs[2]
        USE_COLS_DICT=update_use_cols_dict(USE_COLS_DICT_ORIG)
        self.feat_cols=USE_COLS_DICT[use_cols]
        self.dim=len(self.feat_cols)
        self.data_cols=['datetime']+self.feat_cols
        if self.use_alt_data:dim=self.dim+GDIM
        else:dim=self.dim
        observation_space=spaces.Box(high=np.inf*np.ones(dim),low=-np.inf*np.ones(dim))
        super().__init__(algoclass,monclass=monclass,action_space=action_space,observation_space=observation_space,
                        n_steps=n_steps,verbose=verbose,win=win,soclass=soclass,metarl=metarl)
        self.actor.percept_to_state=self.percept_to_state
        # self.perception.perceive_reward=self.perceive_reward
    def load_model(self,filepath='./saved_models/PPO0.pth'):
        self.model.policy.load_state_dict(torch.load(filepath))
    def percept_to_state(self,perceived_state):
        # self.logL+=[{'printing perceived_state':perceived_state}]
        percept=perceived_state['percept']
        if self.use_alt_data:
            dstate=torch.tensor(percept[0][self.feat_cols].iloc[-1].values)
            gstate=percept[1]
            state=torch.concat((dstate,gstate))
        else: state=torch.tensor(percept[self.feat_cols].iloc[-1].values)
        return state
    def clear_sar_memory(self):
        if len(self.memory.sar_memory)==0: return
        t0=max(self.time-1,0)
        zero_state=np.zeros(self.memory.sar_memory[0]['state'].shape[0])
        for w in range(self.win):
            if self.time-w in self.memory.sar_memory:
                self.memory.sar_memory[self.time-w]['state']=zero_state
                self.memory.sar_memory[self.time-w]['action']=[0,0,0]
                self.memory.sar_memory[self.time-w]['reward']=0
    def check_entry_batch(self,dfD):
        if self.use_alt_data: 
            gfD={t:self.gstateD[dfD[t].iloc[-1]['Date']] if dfD[t].iloc[-1]['Date'] 
                 in self.gstateD else torch.zeros(3) for t in dfD}
        decisionsD,stopD,targetD={},{},{}
        if self.act_on_entry(): 
            limit={0:.5,1:1,2:2}
            trt=self.scantickers[self.tidx]
            df=dfD[trt]
            if self.use_alt_data: action=self.act((df,gfD[trt]))
            else: action=self.act(df)
            decisionsD[trt]=action[0]-1
            stopD[trt]=limit[action[1]]
            targetD[trt]=limit[action[2]]
        return decisionsD,stopD,targetD
    def reward(self,reward):
        if type(reward[0])==list: 
            if len(reward[0])==0: rew=0
            else: rew=reward[0][0]['ppnl']
            reward=(rew,reward[1],reward[2])
        return super().reward(reward)

In [ ]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

In [ ]:
def get_alt_data():
    global feed
    aD={'gdata':feed.gdata}
    return aD